# <span style="color:magenta"> Prepare TE and gene model annotation files. </span>

This notebook combines the TE model data with the gene model data, as they were previously separated.

1. Get the annotations file (.gff3) and the reference file (ctg.fa)
2. SeqIO.parse through reference genome and get a list of all contig names.
3. Make pandas dataframe of annotation file.
4. Subset dataframe to column 0 that only has values in list of contig names from reference file.
5. Use "to.csv(\t)" to convert the subset to a tab-separated GFF file.
6. Cat this subset GFF file of TE data to the gene data GFF file.

In [2]:
import pybedtools
from pybedtools import BedTool
import os
import pysam
import pandas as pd
import glob
import wiggelen
import pybedtools
from pybedtools import BedTool
from Bio import SeqIO

In [4]:
#First we need to define the base dirs
DIRS ={}
DIRS['BASE'] = '/home/anjuni/methylation_calling/pacbio'
DIRS['BED_INPUT'] = os.path.join(DIRS['BASE'], 'input', 'sorted_bed_files')
DIRS['BED_OUT'] = os.path.join(DIRS['BASE'], 'input', 'sorted_bed_files', 'out')

In [5]:
#Quick chech if directories exist
for value in DIRS.values():
    if not os.path.exists(value):
        print('%s does not exist' % value)
    else:
        print(value)

/home/anjuni/methylation_calling/pacbio
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/out


In [6]:
#Make filepaths
bed_file_list = [fn for fn in glob.iglob('%s/*.bed' % DIRS['BED_INPUT'], recursive=True)]

In [7]:
#Check that the list works
print(*bed_file_list, sep='\n')

/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_hc_nanopolish_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_tombo_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_nanopolish_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_tombo_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_smrtlink_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_prob_smrtlink_sorted.bed


In [8]:
# Testing out makign a BedT
nbed = '/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_hc_nanopolish_sorted.bed'

In [12]:
name = (nbed[63:-4])
print(name)

5mC_hc_nanopolish_sorted


In [13]:
name = BedTool(nbed)

In [14]:
print(name)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
# Making a BedTool object for each bed file.
for file in bed_file_list:
    name = (file[63:-4])
    #name = pybedtools.BedTool(file)
    print(name)

5mC_hc_nanopolish_sorted
5mC_tombo_sorted
5mC_nanopolish_sorted
6mA_tombo_sorted
6mA_smrtlink_sorted
6mA_prob_smrtlink_sorted
